In [1]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, LinearColorMapper, LogColorMapper
from bokeh.transform import factor_cmap
from bokeh.palettes import Colorblind, Magma256

output_notebook()
palette = Colorblind[8]
palette_gradient = Magma256

Loading BokehJS ...

In [2]:
df_iterations = pd.read_csv("results.20220312-204209.duet.csv")

In [27]:
df_iterations["iteration_start_ns"] = df_iterations.groupby(["benchmark", "pair"])["iteration_time_ns"].transform(pd.Series.cumsum)
df_iterations["iteration_start_ns"] = df_iterations.groupby(["benchmark", "pair"])["iteration_start_ns"].shift(1, fill_value=0)
df_iterations["iteration_start_ns"] = df_iterations["iteration_start_ns"] + df_iterations["epoch_start_ms"] * 1000

df_iterations["iteration_end_ns"] = df_iterations["iteration_start_ns"] + df_iterations["iteration_time_ns"]

df_iterations.describe()

,wallclock_start_ms,epoch_start_ms,iteration_time_ns,iteration,machine,provider,kind,total_ms,process_cpu_time_ns,compilation_time_ms,compilation_total_ms,iteration_start_ns,iteration_end_ns
count,2.000000e+02,2.000000e+02,2.000000e+02,200.000000,0.0,0.0,0.0,0.0,0.0,0.0,200.000000,2.000000e+02,2.000000e+02
mean,1.647114e+12,1.647114e+12,1.679282e+09,49.500000,NaN,NaN,NaN,NaN,NaN,NaN,68307.000000,1.647203e+15,1.647205e+15
std,1.503764e+00,1.503764e+00,6.679678e+08,28.938507,NaN,NaN,NaN,NaN,NaN,NaN,540.352574,4.624335e+10,4.603212e+10
min,1.647114e+12,1.647114e+12,1.238910e+09,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,67768.000000,1.647114e+15,1.647122e+15
25%,1.647114e+12,1.647114e+12,1.478943e+09,24.750000,NaN,NaN,NaN,NaN,NaN,NaN,67768.000000,1.647164e+15,1.647166e+15
50%,1.647114e+12,1.647114e+12,1.566991e+09,49.500000,NaN,NaN,NaN,NaN,NaN,NaN,68307.000000,1.647203e+15,1.647205e+15
75%,1.647114e+12,1.647114e+12,1.690033e+09,74.250000,NaN,NaN,NaN,NaN,NaN,NaN,68846.000000,1.647243e+15,1.647244e+15
max,1.647114e+12,1.647114e+12,7.922863e+09,99.000000,NaN,NaN,NaN,NaN,NaN,NaN,68846.000000,1.647284e+15,1.647285e+15


In [21]:
source = ColumnDataSource(df_iterations[df_iterations.iteration < 10])
pair_cmap = factor_cmap("pair", palette=palette, factors=sorted(df_iterations["pair"].unique()))
p = figure()
p.circle(x="iteration_start_ns", y="iteration", color=pair_cmap, size=9, source=source)
p.cross(x="iteration_end_ns", y="iteration", color=pair_cmap, size=9, source=source)

show(p)

In [36]:
columns_of_interest = ["benchmark", "pair", "iteration", "iteration_time_ns", "iteration_start_ns", "iteration_end_ns"]
df = df_iterations[columns_of_interest]

# Compute overlap interval
df_overlap = df[df.pair == "A"].merge(df[df.pair == "B"], on=["benchmark"], suffixes=["_A", "_B"], how="inner")
df_overlap["overlap_start_ns"] = df_overlap[["iteration_start_ns_A", "iteration_start_ns_B"]].max(axis=1)
df_overlap["overlap_end_ns"] = df_overlap[["iteration_end_ns_A", "iteration_end_ns_B"]].min(axis=1)

df_overlap.drop(["pair_A", "pair_B"], axis=1, inplace=True)

# Filter out non overlapping intervals
df_overlap["overlap"] = df_overlap["overlap_start_ns"].lt(df_overlap["overlap_end_ns"])
df_overlap = df_overlap[df_overlap["overlap"] == True]
df_overlap.reset_index(drop=True, inplace=True)

# Compute overlap properties
df_overlap["overlap_size"] = df_overlap["overlap_end_ns"] - df_overlap["overlap_start_ns"]
df_overlap["overlap_proportion_A"] = df_overlap["overlap_size"] / df_overlap["iteration_time_ns_A"]
df_overlap["overlap_proportion_B"] = df_overlap["overlap_size"] / df_overlap["iteration_time_ns_B"]
df_overlap["iteration_difference_time_ns"] = df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]
df_overlap["iteration_difference_time_ns_size_weighted"] = (df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]) * df_overlap["overlap_size"]
df_overlap["iteration_difference_time_ns_proportion_weighted"] = (df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]) * df_overlap["overlap_proportion_A"]
df_overlap["iteration_difference_time_ns_scaled"] = (df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]) / df_overlap["iteration_time_ns_A"]
df_overlap["iteration_difference_time_ns_scaled_size_weighted"] = ((df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]) / df_overlap["iteration_time_ns_A"]) * df_overlap["overlap_size"]
df_overlap["iteration_difference_time_ns_scaled_proportion_weighted"] = ((df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]) / df_overlap["iteration_time_ns_A"]) * df_overlap["overlap_proportion_A"]
df_overlap


,benchmark,iteration_A,iteration_time_ns_A,iteration_start_ns_A,iteration_end_ns_A,iteration_B,iteration_time_ns_B,iteration_start_ns_B,iteration_end_ns_B,overlap_start_ns,...,overlap,overlap_size,overlap_proportion_A,overlap_proportion_B,iteration_difference_time_ns,iteration_difference_time_ns_size_weighted,iteration_difference_time_ns_proportion_weighted,iteration_difference_time_ns_scaled,iteration_difference_time_ns_scaled_size_weighted,iteration_difference_time_ns_scaled_proportion_weighted
0,chi-square,0,7922862709,1647113929416000,1647121852278709,0,7904172022,1647113929413000,1647121833585022,1647113929416000,...,True,7904169022,0.997641,1.000000,18690687,147734349185298114,1.864659e+07,0.002359,1.864659e+07,0.002354
1,chi-square,0,7922862709,1647113929416000,1647121852278709,1,2637168310,1647121833585022,1647124470753332,1647121833585022,...,True,18693687,0.002359,0.007089,5285694399,98809116672559113,1.247139e+07,0.667145,1.247139e+07,0.001574
2,chi-square,1,2797235407,1647121852278709,1647124649514116,1,2637168310,1647121833585022,1647124470753332,1647121852278709,...,True,2618474623,0.936094,0.992911,160067097,419131631471779431,1.498378e+08,0.057223,1.498378e+08,0.053566
3,chi-square,1,2797235407,1647121852278709,1647124649514116,2,2858888130,1647124470753332,1647127329641462,1647124470753332,...,True,178760784,0.063906,0.062528,-61652723,-11021089099214832,-3.939993e+06,-0.022041,-3.939993e+06,-0.001409
4,chi-square,2,2298627066,1647124649514116,1647126948141182,2,2858888130,1647124470753332,1647127329641462,1647124649514116,...,True,2298627066,1.000000,0.804028,-560261064,-1287831245736358224,-5.602611e+08,-0.243737,-5.602611e+08,-0.243737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,chi-square,97,1508433174,1647274479140290,1647275987573464,93,1711141048,1647274941466040,1647276652607088,1647274941466040,...,True,1046107424,0.693506,0.611351,-202707874,-212054211894656576,-1.405791e+08,-0.134383,-1.405791e+08,-0.093195
191,chi-square,98,1403494334,1647275987573464,1647277391067798,93,1711141048,1647274941466040,1647276652607088,1647275987573464,...,True,665033624,0.473841,0.388649,-307646714,-204595409123111536,-1.457757e+08,-0.219201,-1.457757e+08,-0.103866
192,chi-square,98,1403494334,1647275987573464,1647277391067798,94,1549734889,1647276652607088,1647278202341977,1647276652607088,...,True,738460710,0.526159,0.476508,-146240555,-107992904076094050,-7.694574e+07,-0.104197,-7.694574e+07,-0.054824
193,chi-square,99,1410932026,1647277391067798,1647278801999824,94,1549734889,1647276652607088,1647278202341977,1647277391067798,...,True,811274179,0.574992,0.523492,-138802863,-112607178723174477,-7.981049e+07,-0.098377,-7.981049e+07,-0.056566


In [39]:
df_overlap[[
    "iteration_time_ns_A",
    "iteration_time_ns_B",
    "overlap_size",
    "iteration_difference_time_ns",
    "iteration_difference_time_ns_size_weighted",
    "iteration_difference_time_ns_proportion_weighted",
    "iteration_difference_time_ns_scaled_size_weighted",
    "iteration_difference_time_ns_scaled_proportion_weighted"]].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
iteration_time_ns_A,195.0,1.651581e+09,6.713841e+08,1.346566e+09,1.463379e+09,1.530734e+09,1.663132e+09,7.922863e+09
iteration_time_ns_B,195.0,1.701885e+09,5.108745e+08,1.391681e+09,1.533391e+09,1.619108e+09,1.721210e+09,7.904172e+09
overlap_size,195.0,8.455004e+08,7.166261e+08,7.692777e+06,3.827330e+08,8.068349e+08,1.237125e+09,7.904169e+09
iteration_difference_time_ns,195.0,-5.030317e+07,4.311763e+08,-1.124675e+09,-1.780712e+08,-7.213604e+07,3.814306e+07,5.285694e+09
iteration_difference_time_ns_size_weighted,195.0,-6.617413e+16,2.022227e+17,-1.287831e+18,-1.337369e+17,-3.338420e+16,1.230596e+16,5.327773e+17
iteration_difference_time_ns_proportion_weighted,195.0,-4.547176e+07,1.152680e+08,-5.714352e+08,-8.924367e+07,-2.290962e+07,7.667832e+06,2.553352e+08
iteration_difference_time_ns_scaled_size_weighted,195.0,-4.547176e+07,1.152680e+08,-5.714352e+08,-8.924367e+07,-2.290962e+07,7.667832e+06,2.553352e+08
iteration_difference_time_ns_scaled_proportion_weighted,195.0,-3.090101e-02,6.933707e-02,-3.541545e-01,-6.181897e-02,-1.392176e-02,3.776182e-03,1.256095e-01


In [6]:
source = ColumnDataSource(df_overlap)
p = figure()
p.hbar(y="index", left="overlap_start_ns", right="overlap_end_ns", source=source,
       color={"field": "overlap_size", "transform": LogColorMapper(palette=palette_gradient)})

show(p)

In [7]:
source = ColumnDataSource(df_overlap)
p = figure()
p.vbar(x="overlap_start_ns", top="overlap_size", source=source)

show(p)